### Import

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import *
from vvm_var_func import *

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn.colors.xkcd_rgb as c
import cmaps
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### Case assigning

In [3]:
case_name = 'pbl_mod_tracer_chem'

### Load multi-timestep varfiles

In [4]:
# Get varname
var_fname, var_longname = create_var_dict(case_name)

In [7]:
var_fname

{'zc': 'Dynamic',
 'zb': 'Radiation',
 'yc': 'Surface',
 'xc': 'Surface',
 'fusw': 'Radiation',
 'fdsw': 'Radiation',
 'fulw': 'Radiation',
 'fdlw': 'Radiation',
 'dtradsw': 'Radiation',
 'dtradlw': 'Radiation',
 'cldfrc': 'Radiation',
 'fuswtoa': 'Radiation',
 'fdswtoa': 'Radiation',
 'fulwtoa': 'Radiation',
 'emissi': 'LandSurface',
 'cmc': 'LandSurface',
 't1': 'LandSurface',
 'ST1': 'LandSurface',
 'ST2': 'LandSurface',
 'ST3': 'LandSurface',
 'ST4': 'LandSurface',
 'SM1': 'LandSurface',
 'SM2': 'LandSurface',
 'SM3': 'LandSurface',
 'SM4': 'LandSurface',
 'SL1': 'LandSurface',
 'SL2': 'LandSurface',
 'SL3': 'LandSurface',
 'SL4': 'LandSurface',
 'snowh': 'LandSurface',
 'sneqv': 'LandSurface',
 'albedo': 'LandSurface',
 'ch': 'LandSurface',
 'cm': 'LandSurface',
 'eta': 'Dynamic',
 'fdown': 'LandSurface',
 'ec': 'LandSurface',
 'edir': 'LandSurface',
 'et1': 'LandSurface',
 'et2': 'LandSurface',
 'et3': 'LandSurface',
 'et4': 'LandSurface',
 'ett': 'LandSurface',
 'esnow': 'LandSu

In [6]:
var_longname

{'vertical height of model layers, MSL': 'zc',
 'vertical height of model interfaces, MSL': 'zb',
 'y-coordinate of grid cell centers in Cartesian system': 'yc',
 'x-coordinate of grid cell centers in Cartesian system': 'xc',
 'Upward flux of shortwave radiation': 'fusw',
 'Downward flux of shortwave radiation': 'fdsw',
 'Upward flux of longwave radiation': 'fulw',
 'Downward flux of longwave radiation': 'fdlw',
 'Net shortwave heating rate': 'dtradsw',
 'Net longwave heating rate': 'dtradlw',
 'Cloud fraction': 'cldfrc',
 'Upward flux of shortwave radiation at TOA': 'fuswtoa',
 'Downward flux of shortwave radiation at TOA': 'fdswtoa',
 'Upward flux of longwave radiation at TOA': 'fulwtoa',
 'emissivity': 'emissi',
 'canopy moisture content': 'cmc',
 'skin temperature': 't1',
 'soil temperature layer 1': 'ST1',
 'soil temperature layer 2': 'ST2',
 'soil temperature layer 3': 'ST3',
 'soil temperature layer 4': 'ST4',
 'soil moisture layer 1': 'SM1',
 'soil moisture layer 2': 'SM2',
 's

In [8]:
ds_dyn = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/{case_name}/archive/{case_name}.L.Dynamic*')

In [11]:
ds_thm = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/{case_name}/archive/{case_name}.L.Thermodynamic*')

In [12]:
ds_chem= xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/{case_name}/archive/{case_name}.L.Chemical*')

In [13]:
ds_tra = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/{case_name}/archive/{case_name}.L.Tracer*')

### PBL_boundary

In [14]:
u, v, w = ds_dyn.u, ds_dyn.v, ds_dyn.w

In [15]:
th      = ds_thm.th

#### lat-averaged TKE

In [46]:
tke   = (u**2+v**2+w**2).mean(dim=['lat'])

In [47]:
arr_tke = tke.values

In [48]:
arr_tke.shape

(721, 50, 128)

#### lat-averaged Enstrophy

In [18]:
xi, eta, zeta = ds_dyn.xi, ds_dyn.eta, ds_dyn.zeta

In [59]:
enst   = (xi**2+eta**2+zeta**2).mean(dim=['lat'])

In [60]:
arr_enst = enst.values

#### Height where theta=theta(lev=0)+0.5K

In [22]:
def find_level(pt, levels):
    # Iterate over levels and find the first level where pt >= pt[lev0] + 0.5
    target_temp = pt[0] + 0.5
    idx = np.argmax(pt >= target_temp, axis=0)
    
    # Set to 0 if no level satisfies the condition
    idx = np.where(np.any(pt >= target_temp, axis=0), idx, 0)
    return levels[idx]

In [61]:
th05 = xr.apply_ufunc(
    find_level,
    th,
    th.lev,
    input_core_dims=[['lev'], ['lev']],
    vectorize=True,  # Apply across all other dimensions
    dask="parallelized",  # Use Dask for larger datasets
    output_dtypes=[float]  # Output will be float
).mean(dim=['lat'])

In [62]:
arr_th05 = th05.values

In [67]:
arr_th05.shape

(721, 128)

#### d_theta/dz max

In [25]:
arr_th   = th.values

In [26]:
arr_th_slope = (arr_th[:, 1:, ...]-arr_th[:, :-1, ...])/0.02

In [27]:
arr_slope_idx   = np.argmax(arr_th_slope, axis=1)

In [28]:
slope_idx   = xr.DataArray(arr_slope_idx, dims=['time', 'lat', 'lon'])

In [63]:
dthdz_H   = th.lev.isel(lev=slope_idx).mean(dim=['lat'])

In [64]:
arr_dthdz_H   = dthdz_H.values

In [66]:
arr_dthdz_H.shape

(721, 128)

### Tracer

In [31]:
tr01, tr02, tr03 = ds_tra.tr01, ds_tra.tr02, ds_tra.tr03

#### Tracer 1 (initiate level index=8)

In [167]:
def Plot_tr01_ymean(time:int, figpath=False):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.grid(':', linewidth=0.5)
    x      = np.arange(tke.lon.shape[0])
    # TKE
    #print(arr_tke[time, ...].max())
    cs     = ax.contour(x, tke.lev, arr_tke[time, ...], 
                        levels=[0.25], colors='w', linewidths=3)
    ax.clabel(cs, fmt='%4.2f', fontsize=12)
    # Theta+0.5
    ax.plot(x, arr_th05[time, :], 'k', linewidth=3, label=r'$\theta$(z=0) +0.5K')
    # max. theta variability
    ax.plot(x, arr_dthdz_H[time, :], color='grey', linewidth=3, label=r'd$\theta$/dz max.')
    
    # Tracer 01
    bounds2= np.arange(0, 0.141, 0.02)
    im     = ax.contourf(x, tr01.lev, tr01.isel(time=time).mean(dim='lat'), 
                         levels=bounds2, cmap=cmaps.cmocean_amp, extend='max')
    cbar   = fig.colorbar(im, extend='max')
    cbar.set_ticks(ticks=bounds2, labels=bounds2)
    cbar.ax.tick_params(labelsize=13)
    cbar.set_label('kg/kg', fontsize=14)
    # Info settings
    ax.set_xticks(np.arange(0, x.shape[0]+0.1, 32))
    ax.set_xticklabels(np.arange(0, 256.1, 64).astype(int), fontsize=13)
    ax.set_xlabel('x (km)', fontsize=14)
    ax.set_yticks(np.arange(0, 1.9, 0.25))
    ax.set_yticklabels(np.arange(0, 1.9, 0.25), fontsize=13)
    ax.set_ylabel('Height (km)', fontsize=14)
    ax.legend(loc='upper right', fontsize=14)
    ax.set_title('Tracer 01', fontsize=16)
    ax.set_title('y-mean', loc='right', fontsize=12)
    ax.set_title(f'Time = {(datetime(2024, 1, 1, 5)+timedelta(seconds=120*time)).strftime("%H:%M:%S")}', 
                 loc='left', fontsize=12)
    if figpath:
        plt.savefig(f'./Figure/{figpath}/timestep_{time:03d}.png', bbox_inches='tight',
                    facecolor='w', dpi=400)
    else:
        plt.show()

In [ ]:
for i in range(tr01.time.shape[0]):
    Plot_tr01_ymean(time=i, figpath='hw4_tracer01')

In [ ]:
!ffmpeg -r 10 -f image2 -y -pattern_type glob -i './Figure/hw4_tracer01/*.png' -q:v 1 -vcodec libx264 -crf 25 -pix_fmt yuv420p -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" ./Figure/hw4_tracer01/hw4_tr01_ymean.mp4

#### Tracer 02

In [171]:
def Plot_tr02_ymean(time:int, figpath=False):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.grid(':', linewidth=0.5)
    x      = np.arange(tke.lon.shape[0])
    # TKE
    cs     = ax.contour(x, tke.lev, arr_tke[time, ...], 
                        levels=[0.25], colors='w', linewidths=3)
    ax.clabel(cs, fmt='%4.2f', fontsize=12)
    # Enstrophy
    cs2    = ax.contour(x, enst.lev, arr_enst[time, ...], 
                        levels=[2e-5], colors=c['denim'], linewidths=3)
    #ax.clabel(cs2, fmt='%4.2f', fontsize=12)
    # Theta+0.5
    ax.plot(x, arr_th05[time, :], 'k', linewidth=3, label=r'$\theta$(z=0) +0.5K')
    # max. theta variability
    ax.plot(x, arr_dthdz_H[time, :], color='grey', linewidth=3, label=r'd$\theta$/dz max.')
    
    # Tracer 02
    bounds2= np.arange(0, 5.1, 0.5)
    im     = ax.contourf(x, tr02.lev, tr02.isel(time=time).mean(dim='lat'), 
                         levels=bounds2, cmap=cmaps.cmocean_amp, extend='max')
    cbar   = fig.colorbar(im, extend='max')
    cbar.set_ticks(ticks=bounds2, labels=bounds2)
    cbar.ax.tick_params(labelsize=13)
    cbar.set_label('kg/kg', fontsize=14)
    # Info settings
    ax.set_xticks(np.arange(0, x.shape[0]+0.1, 32))
    ax.set_xticklabels(np.arange(0, 256.1, 64).astype(int), fontsize=13)
    ax.set_xlabel('x (km)', fontsize=14)
    ax.set_yticks(np.arange(0, 1.9, 0.25))
    ax.set_yticklabels(np.arange(0, 1.9, 0.25), fontsize=13)
    ax.set_ylabel('Height (km)', fontsize=14)
    ax.legend(loc='upper right', fontsize=14)
    ax.set_title('Tracer 02', fontsize=16)
    ax.set_title('y-mean', loc='right', fontsize=12)
    ax.set_title(f'Time = {(datetime(2024, 1, 1, 5)+timedelta(seconds=120*time)).strftime("%H:%M:%S")}', 
                 loc='left', fontsize=12)
    if figpath:
        plt.savefig(f'./Figure/{figpath}/timestep_{time:03d}.png', bbox_inches='tight', 
                    facecolor='w', dpi=400)
    else:
        plt.show()

In [ ]:
for i in range(tr02.time.shape[0]):
    Plot_tr02_ymean(time=i, figpath='hw4_tracer02')

In [ ]:
!ffmpeg -r 10 -f image2 -y -pattern_type glob -i './Figure/hw4_tracer02/*.png' -q:v 1 -vcodec libx264 -crf 25 -pix_fmt yuv420p -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" ./Figure/hw4_tracer02/hw4_tr02_ymean.mp4